In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/internet-articles-data-with-users-engagement/articles_data.csv',
                 usecols=['source_name','author','title','description','published_at','top_article','engagement_reaction_count',
                           'engagement_comment_count','engagement_share_count'])
df.head()

In [ ]:
df.isna().sum()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import string
import re
from wordcloud import WordCloud,STOPWORDS
import nltk
from sklearn.metrics import mean_squared_error
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer as CVTZ
from sklearn.linear_model import LinearRegression
from keras import Sequential,Model
from keras.layers import Dense
from keras.layers.merge import concatenate
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf
import collections
def set_seed(seed=12345):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

def RMSE(Y,YHAT):
    return np.sqrt(mean_squared_error(Y,YHAT))

plt.rc('figure',figsize=(20,11))

In [ ]:
df.published_at = pd.to_datetime(df.published_at)

df['Day_Of_Week'] = df.published_at.apply(lambda x: x.dayofweek)
df['Month'] = df.published_at.apply(lambda x: x.month)
df['Year'] = df.published_at.apply(lambda x: x.year)

In [ ]:
plt.subplot(2,1,1)
plt.title("Reaction Counts Before Log Transformation")
sns.kdeplot(df.engagement_reaction_count)
plt.subplot(2,1,2)
plt.title("Reaction Counts After Log Transformation")
sns.kdeplot(np.log(df.engagement_reaction_count+0.00001))
df.engagement_reaction_count = np.log(df.engagement_reaction_count+0.00001)

In [ ]:
info = df.describe()
info.loc['Skewness'] = df.skew()
info.loc['Kurtosis'] = df.kurt()
info.loc['Median'] = df.median()
info

In [ ]:
import plotly.express as ex
ex.pie(df,names='top_article',title='"TOP ARTICLE" ')

In [ ]:
from wordcloud import WordCloud
from wordcloud import WordCloud,STOPWORDS
%matplotlib inline

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
def clean_text(text, lemmatizer = WordNetLemmatizer(), 
                  stop_words = set(stopwords.words('english'))):
      
    words = word_tokenize(text)
    
    filtered_words = []
    
    for word in words:
        
        if word not in stop_words and word.isalpha():
            filtered_words.append(lemmatizer.lemmatize(word))
    
    return filtered_words

In [ ]:
words_title = clean_text(''.join(str(df['title'].tolist())))

In [ ]:
word_cloud = WordCloud(background_color="white", 
               max_words=2000,
               relative_scaling='auto',
               colormap='winter')

word_cloud.generate(','.join(words_title))
plt.figure( figsize=(12,10) )
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
words_desc = clean_text(''.join(str(df['description'].tolist())))
(pd.Series(nltk.ngrams(words_desc, 4)).value_counts())[:20]

In [ ]:
fourgrams_series_title = (pd.Series(nltk.ngrams(words_desc, 4)).value_counts())[:20]

fourgrams_series_title.sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))
plt.title('20 Most Frequently Occuring Fourgrams')
plt.ylabel('fourgram')
plt.xlabel('# of Occurances')

In [ ]:
trigrams_series_title = (pd.Series(nltk.ngrams(words_desc, 3)).value_counts())[:10]

trigrams_series_title.sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))
plt.title('10 Most Frequently Occuring Trigrams')
plt.ylabel('trigram')
plt.xlabel('# of Occurances')

In [ ]:
word_cloud = WordCloud(background_color="white", 
               max_words=2000,
               relative_scaling='auto',
               colormap='winter')

word_cloud.generate(','.join(words_desc))
plt.figure( figsize=(12,10) )
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis('off')
plt.show()